In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import cv2 as cv

1.3

In [3]:
base_dir = "Jute_Pest_Dataset"
batch_size = 32
img_height = 224
img_width = 224

# Define directories for training, validation, and testing datasets
train_dir = f'{base_dir}/train'
val_dir = f'{base_dir}/val'
test_dir = f'{base_dir}/test'

# Load training dataset with specified image size and batch size
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  train_dir,
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

# Store class names for reference
class_names = train_ds.class_names

# Load validation dataset
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  val_dir,
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

# Load test dataset
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  test_dir,
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)


Found 6443 files belonging to 17 classes.
Found 413 files belonging to 17 classes.
Found 379 files belonging to 17 classes.


In [4]:
for images, labels in train_ds.take(1):  # Taking a single batch
    print(images.numpy().shape)  # Accessing the feature tensors (images)
    print(labels.numpy())  # Accessing the integer labels
    print([class_names[label] for label in labels.numpy()])  # Convert integer labels to class names

(32, 224, 224, 3)
[ 9  9  6  4 15  9  2  2 11  7  6 15  6 11 15  8  7  7 12 13 13  9 14  6
 10  8 13 16  0  7  2  5]
['Jute Stem Weevil', 'Jute Stem Weevil', 'Jute Red Mite', 'Jute Aphid', 'Termite odontotermes (Rambur)', 'Jute Stem Weevil', 'Cutworm', 'Cutworm', 'Mealybug', 'Jute Semilooper', 'Jute Red Mite', 'Termite odontotermes (Rambur)', 'Jute Red Mite', 'Mealybug', 'Termite odontotermes (Rambur)', 'Jute Stem Girdler', 'Jute Semilooper', 'Jute Semilooper', 'Pod Borer', 'Scopula Emissaria', 'Scopula Emissaria', 'Jute Stem Weevil', 'Termite', 'Jute Red Mite', 'Leaf Beetle', 'Jute Stem Girdler', 'Scopula Emissaria', 'Yellow Mite', 'Beet Armyworm', 'Jute Semilooper', 'Cutworm', 'Jute Hairy']


In [ ]:
# Define data augmentation pipeline with common transformations
data_augmentation = keras.Sequential([
    keras.layers.RandomFlip("horizontal_and_vertical"),  # Randomly flip images horizontally and vertically
    keras.layers.RandomRotation(0.2),                    # Apply random rotation
    keras.layers.RandomZoom(0.2),                        # Apply random zoom
    keras.layers.RandomContrast(0.1)                     # Adjust contrast randomly
])

train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))


1.4

In [5]:
Model = keras.models.Sequential([
    # Convolutional Layer, Batch Normalization, and Max Pooling
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2, 2)),

    # Convolutional Layer, Batch Normalization, and Max Pooling
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2, 2)),

    # Dropout for regularization
    keras.layers.Dropout(0.5),

    # Flatten and Fully Connected Layer with Regularization
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.5),

    # Output Layer for 17-class classification
    keras.layers.Dense(17, activation='softmax')
])


c:\Users\Kehan Manamperi\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
model = Model
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 222, 222, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 109, 109, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 186624)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    23,888,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 17)             │         2,193 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,910,481 (91.21 MB)

 Trainable params: 23,910,033 (91.21 MB)

 Non-trainable params: 448 (1.75 KB)

In [7]:
# Compile the model with Adam optimizer and sparse categorical cross-entropy loss for multi-class classification
model.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_ds, validation_data=val_ds, epochs=20)


Epoch 1/20
202/202 ━━━━━━━━━━━━━━━━━━━━ 218s 1s/step - accuracy: 0.2034 - loss: 4.0733 - val_accuracy: 0.3559 - val_loss: 3.6941
Epoch 2/20
202/202 ━━━━━━━━━━━━━━━━━━━━ 206s 1s/step - accuracy: 0.3873 - loss: 3.5278 - val_accuracy: 0.3753 - val_loss: 3.8938
Epoch 3/20
202/202 ━━━━━━━━━━━━━━━━━━━━ 209s 1s/step - accuracy: 0.4837 - loss: 3.4593 - val_accuracy: 0.2833 - val_loss: 4.4668
Epoch 4/20
202/202 ━━━━━━━━━━━━━━━━━━━━ 203s 1s/step - accuracy: 0.5777 - loss: 3.6021 - val_accuracy: 0.4600 - val_loss: 4.6609
Epoch 5/20
202/202 ━━━━━━━━━━━━━━━━━━━━ 211s 1s/step - accuracy: 0.6516 - loss: 3.9725 - val_accuracy: 0.4479 - val_loss: 5.3229
Epoch 6/20
202/202 ━━━━━━━━━━━━━━━━━━━━ 214s 1s/step - accuracy: 0.7249 - loss: 4.2902 - val_accuracy: 0.5085 - val_loss: 5.6500
Epoch 7/20
202/202 ━━━━━━━━━━━━━━━━━━━━ 221s 1s/step - accuracy: 0.7813 - loss: 4.4476 - val_accuracy: 0.5932 - val_loss: 5.8660
Epoch 8/20
202/202 ━━━━━━━━━━━━━━━━━━━━ 226s 1s/step - accuracy: 0.8078 - loss: 4.7115 - val_accu